In [2]:
HTML(read(open("style.css"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

# Memoization

This notebook contains alle functions needed for memoizing function calls that we have already calculated. This will greatly reduce the runtime of the program.  

```
function memoize(f)
    global gCache
    function f_memoized(b, s, d)    # TODO: replace (b, s, d) with args... to make function generic
        if (b, s, d) in keys(gCache)
            return gCache[args]
        end
        result = f(b, s, d)
        merge!(gCache, Dict((b, s, d) => result))
        return result
    end
    return f_memoized
end
```

In [3]:
using Chess
using NBInclude

In [4]:
@nbinclude("ZobristHashing.ipynb")

zobrist_hash (generic function with 2 methods)

The function `initCache()` returns an empty Dictionary of type `Dict{UInt64, Tuple{String, Int64, Int64}}` which will be used for the global Cache variable.

In [5]:
function initCache()
    return Dict{UInt64, Tuple{String, Int64, Int64}}()
end

initCache (generic function with 1 method)

The `evaluate` function adds memoization to the `Max` and `Min` function.

It takes in 4 arguments and 2 optional arguments.
1. `State` is the current state represented by a `Board`
1. `f` takes in either the function `alphaBetaMax` and `alphaBetaMin`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process

The function `evaluate` evaluates the same result as the function `f`. Additionally it saves calculated results in the `Cache`. And uses any entries in the `Cache` if the same function has already been called.

In [4]:
function evaluate(State::Board, f::Function, score::Int64, hash::UInt64, depth::Int64, cache:: Dict{UInt64, Tuple{String, Int64, Int64}},
                  alpha::Int64=-200000, beta::Int64=200000)::Int64
    if hash in keys(cache)
        flag, v, d = cache[hash]
        if d >= depth   # check if stored depth to this position is already higher than required
            if flag == "="
                return v
            elseif flag == "<="
                if v <= alpha
                    return v
                else
                    beta = min(beta, v)
                end
            elseif flag == ">="
                if beta <= v
                    return v
                else
                    alpha = max(alpha, v)
                end
            end
        end
    end
    # no value stored in gCache for State or depth of stored State has less depth than required
    v = f(State, score, hash, depth, cache, alpha, beta)
    store_cache(hash, depth, cache, alpha, beta, v)
    return v
end

evaluate (generic function with 4 methods)

The `store_cache` is a helping function that stores values into the `Cache`. 

It takes 5 arguments:
1. `State` is the current `state` of type `Board`
1. `depth` is the number of halfmoves the engine should analyze before terminating
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process
1. `v` is the value that got calculated by the `evaluate` function that needs to be stored


In [6]:
function store_cache(hash::UInt64, depth::Int64, cache:: Dict{UInt64, Tuple{String, Int64, Int64}},alpha::Int64, beta::Int64, v::Int64)
    if v <= alpha
        cache[hash] = ("<=", v, depth)
    elseif v < beta
        cache[hash] = ("=", v, depth)
    else # beta <= v
        cache[hash] = (">=", v, depth)
    end
end

store_cache (generic function with 2 methods)

In [8]:
function updateBoardData(board::Board, score::Int64, hash::UInt64, move::Move)
    next_score = evaluate_move(board, move, score)
    next_hash = zobrist_hash(board, hash, move)
    return next_score, next_hash
end

updateBoardData (generic function with 1 method)